In [65]:
import pandas as pd

In [66]:
df = pd.read_csv('df_collision_roni.csv')

In [67]:
df

,id,state,video_ref,signal_refs,signals_storage,signals_bucket,storage,bucket,video_type,incident_type,timestamp_millis,duration_seconds,nvidia_delivery,signed_url
0,3b577ab5-df1d-4ee0-be14-cc292fa19a8a,California,user/0faf6e62fe4e291aaec788f94653b78b/ride/5f1...,['signals/dense/ride_id=5f1153b5e234b041365708...,s3,nexar-data-warehouse,s3,nexar-upload,nv_near-collision_full_annotation,AUTO_HARD_BRAKE,1626489641000,40.315000,1,https://nexar-upload.s3.amazonaws.com/user/0fa...
1,incident-0-b43d94ea-958a-4370-b62c-9071baf662c0,Ohio,cognitoId/us-east-1:be7665ee-1450-4884-8f29-45...,['signals/dense/ride_id=56b723c2d8e32789533780...,s3,nexar-data-warehouse,s3,nexar-upload,nv_collision_full_annotation,AUTO_DEVICE_HIT,1638397697000,39.977000,2,https://nexar-upload.s3.amazonaws.com/cognitoI...
2,c9987124-6514-4288-ba1c-e7a1db96b174,California,user/b53c87ae0cffb03b6812e1f0671f52f0/ride/c58...,['signals/dense/ride_id=c580bbb886fddb55404705...,s3,nexar-data-warehouse,s3,nexar-upload,nv_near-collision_full_annotation,AUTO_HARD_BRAKE,1597270995000,40.050000,1,https://nexar-upload.s3.amazonaws.com/user/b53...
3,38fe4bd7c206e093e903537a5b0fa7ae,Texas,Waymo_incidents_iOS_500K/38fe4bd7c206e093e9035...,['signals/dense/ride_id=d54294932ca2c543d1db9c...,s3,nexar-data-warehouse,s3,deep-learning-annotation,nv_near-collision_full_annotation,AUTO_HARD_BRAKE,1613253950000,40.165299,1,https://deep-learning-annotation.s3.amazonaws....
4,8c7fb032-dbf6-4409-baa4-8ef360f44bf5,Illinois,user/d0feb6760e559ea569a3af3cc6dd361e/ride/270...,['signals/dense/ride_id=270d085ae4ea4d2c736053...,s3,nexar-data-warehouse,s3,nexar-upload,nv_collision_full_annotation,AUTO_HARD_BRAKE,1665114649000,40.210000,2,https://nexar-upload.s3.amazonaws.com/user/d0f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21775,49cb2ea106553fad1b0b262be7ab831b,California,Waymo_incidents_iOS_500K/49cb2ea106553fad1b0b2...,['signals/dense/ride_id=76cd47f23e6aba6a7d65a7...,s3,nexar-data-warehouse,s3,deep-learning-annotation,nv_collision_full_annotation,AUTO_HARD_BRAKE,1650385820000,39.233333,2,https://deep-learning-annotation.s3.amazonaws....
21776,incident-0-a81d140b-3640-403b-9518-017f7512917e,California,cognitoId/us-east-1:c6abc023-f156-4fd4-a9bb-77...,['cognitoId/us-east-1:c6abc023-f156-4fd4-a9bb-...,s3,nexar-upload,s3,nexar-upload,nv_collision_full_annotation,AUTO_HARD_BRAKE,1695302076000,38.450000,2,https://nexar-upload.s3.amazonaws.com/cognitoI...
21777,6ca4f576-137f-4389-af7f-160e4cacbc09,New York,user/800f434618a3713782412cf0623843c2/ride/416...,['user/800f434618a3713782412cf0623843c2/ride/4...,s3,nexar-upload,s3,nexar-upload,nv_collision_full_annotation,AUTO_HARD_BRAKE,1555559053000,40.626667,2,https://nexar-upload.s3.amazonaws.com/user/800...
21778,ffbbb385-8350-482c-826d-f084cd769b32,New York,user/10bfec6a8627d60f077d7762ef2307c5/ride/9c2...,['signals/dense/ride_id=9c21dc66c200059e4d6665...,s3,nexar-data-warehouse,s3,nexar-upload,nv_collision_full_annotation,AUTO_HARD_BRAKE,1623401010000,40.498333,2,https://nexar-upload.s3.amazonaws.com/user/10b...


In [68]:
df['video_type'].value_counts()

video_type
nv_collision_full_annotation         12920
nv_near-collision_full_annotation     8860
Name: count, dtype: int64

In [69]:
import boto3
from botocore.client import Config
from datetime import datetime, timedelta
from tqdm import tqdm

BUCKET_TO_REGION = {
    'nexar-upload': 'us-east-1',
    'deep-learning-annotation': 'us-west-2',
}

s3 = boto3.client('s3', region_name='us-east-1', config=Config(signature_version='s3v4'))

def generate_presigned_url(bucket_name, key, expiration_days=7):
    region = BUCKET_TO_REGION[bucket_name]
    s3 = boto3.client('s3', region_name=region, config=Config(signature_version='s3v4'))
    return s3.generate_presigned_url(
        ClientMethod='get_object',
        Params={
            'Bucket': bucket_name,
            'Key': key,
            'ResponseContentType': 'video/mp4',
        },
        ExpiresIn=expiration_days * 24 * 60 * 60,
        HttpMethod='GET'
    )


urls = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Generating S3 URLs"):
    bucket_name = row['bucket']
    key = row['video_ref']
    url = generate_presigned_url(bucket_name, key)
    urls.append(url)

df['signed_url'] = urls

Generating S3 URLs: 100%|██████████| 21780/21780 [01:45<00:00, 206.51it/s]


In [70]:
df.to_csv('df_collision_roni.csv',index=False)

In [71]:
def get_bucket_region(bucket_name):
    s3 = boto3.client('s3')
    region = s3.get_bucket_location(Bucket=bucket_name)['LocationConstraint']
    return region or 'us-east-1'

print(get_bucket_region('deep-learning-annotation'))

us-west-2


In [72]:
df['bucket'].value_counts()

bucket
nexar-upload                13458
deep-learning-annotation     8322
Name: count, dtype: int64

In [73]:
df[['id', 'signed_url']].to_csv('collisions_annotations_final.csv',index=False)

In [74]:
df

,id,state,video_ref,signal_refs,signals_storage,signals_bucket,storage,bucket,video_type,incident_type,timestamp_millis,duration_seconds,nvidia_delivery,signed_url
0,3b577ab5-df1d-4ee0-be14-cc292fa19a8a,California,user/0faf6e62fe4e291aaec788f94653b78b/ride/5f1...,['signals/dense/ride_id=5f1153b5e234b041365708...,s3,nexar-data-warehouse,s3,nexar-upload,nv_near-collision_full_annotation,AUTO_HARD_BRAKE,1626489641000,40.315000,1,https://nexar-upload.s3.amazonaws.com/user/0fa...
1,incident-0-b43d94ea-958a-4370-b62c-9071baf662c0,Ohio,cognitoId/us-east-1:be7665ee-1450-4884-8f29-45...,['signals/dense/ride_id=56b723c2d8e32789533780...,s3,nexar-data-warehouse,s3,nexar-upload,nv_collision_full_annotation,AUTO_DEVICE_HIT,1638397697000,39.977000,2,https://nexar-upload.s3.amazonaws.com/cognitoI...
2,c9987124-6514-4288-ba1c-e7a1db96b174,California,user/b53c87ae0cffb03b6812e1f0671f52f0/ride/c58...,['signals/dense/ride_id=c580bbb886fddb55404705...,s3,nexar-data-warehouse,s3,nexar-upload,nv_near-collision_full_annotation,AUTO_HARD_BRAKE,1597270995000,40.050000,1,https://nexar-upload.s3.amazonaws.com/user/b53...
3,38fe4bd7c206e093e903537a5b0fa7ae,Texas,Waymo_incidents_iOS_500K/38fe4bd7c206e093e9035...,['signals/dense/ride_id=d54294932ca2c543d1db9c...,s3,nexar-data-warehouse,s3,deep-learning-annotation,nv_near-collision_full_annotation,AUTO_HARD_BRAKE,1613253950000,40.165299,1,https://deep-learning-annotation.s3.amazonaws....
4,8c7fb032-dbf6-4409-baa4-8ef360f44bf5,Illinois,user/d0feb6760e559ea569a3af3cc6dd361e/ride/270...,['signals/dense/ride_id=270d085ae4ea4d2c736053...,s3,nexar-data-warehouse,s3,nexar-upload,nv_collision_full_annotation,AUTO_HARD_BRAKE,1665114649000,40.210000,2,https://nexar-upload.s3.amazonaws.com/user/d0f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21775,49cb2ea106553fad1b0b262be7ab831b,California,Waymo_incidents_iOS_500K/49cb2ea106553fad1b0b2...,['signals/dense/ride_id=76cd47f23e6aba6a7d65a7...,s3,nexar-data-warehouse,s3,deep-learning-annotation,nv_collision_full_annotation,AUTO_HARD_BRAKE,1650385820000,39.233333,2,https://deep-learning-annotation.s3.amazonaws....
21776,incident-0-a81d140b-3640-403b-9518-017f7512917e,California,cognitoId/us-east-1:c6abc023-f156-4fd4-a9bb-77...,['cognitoId/us-east-1:c6abc023-f156-4fd4-a9bb-...,s3,nexar-upload,s3,nexar-upload,nv_collision_full_annotation,AUTO_HARD_BRAKE,1695302076000,38.450000,2,https://nexar-upload.s3.amazonaws.com/cognitoI...
21777,6ca4f576-137f-4389-af7f-160e4cacbc09,New York,user/800f434618a3713782412cf0623843c2/ride/416...,['user/800f434618a3713782412cf0623843c2/ride/4...,s3,nexar-upload,s3,nexar-upload,nv_collision_full_annotation,AUTO_HARD_BRAKE,1555559053000,40.626667,2,https://nexar-upload.s3.amazonaws.com/user/800...
21778,ffbbb385-8350-482c-826d-f084cd769b32,New York,user/10bfec6a8627d60f077d7762ef2307c5/ride/9c2...,['signals/dense/ride_id=9c21dc66c200059e4d6665...,s3,nexar-data-warehouse,s3,nexar-upload,nv_collision_full_annotation,AUTO_HARD_BRAKE,1623401010000,40.498333,2,https://nexar-upload.s3.amazonaws.com/user/10b...
